# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [1]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [2]:
html=requests.get(url).content
html[0:100]

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title'

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [3]:
from bs4 import BeautifulSoup as bs

In [4]:
soup=bs(html, 'html.parser')


In [5]:
text = soup.find_all('div', {'class':'mw-parser-output'})


In [6]:
links = text[0].find_all('a')
links

[<a href="/wiki/Information_science" title="Information science">information science</a>,
 <a href="/wiki/Machine_learning" title="Machine learning">Machine learning</a>,
 <a href="/wiki/Data_mining" title="Data mining">data mining</a>,
 <a class="image" href="/wiki/File:Kernel_Machine.svg"><img alt="Kernel Machine.svg" data-file-height="233" data-file-width="512" decoding="async" height="100" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Kernel_Machine.svg/220px-Kernel_Machine.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Kernel_Machine.svg/330px-Kernel_Machine.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Kernel_Machine.svg/440px-Kernel_Machine.svg.png 2x" width="220"/></a>,
 <a href="/wiki/Statistical_classification" title="Statistical classification">Classification</a>,
 <a href="/wiki/Cluster_analysis" title="Cluster analysis">Clustering</a>,
 <a href="/wiki/Regression_analysis" title="Regression analysis">Regression</a>,
 <a h

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [7]:
domain = 'http://wikipedia.org'
import re

In [8]:
def clean(s):
    c = re.findall('ref=\"\/?\/?www.[^"]{1,}"\s',s)
    if c != []:
        s = str(c[0]).replace('href=\"','').replace('"','').rstrip()
    else:
    
        b = re.findall('href=\"https?:[^"]{1,}"',s)
        if b !=[]:
            s = str(b[0]).replace('href=\"','').replace('"','').rstrip()
        else:
            a = re.findall('href=\"[^"]{1,}"\s',s)
            if a != []:
                s = str(a[0]).replace('href="', domain).replace('"', '').rstrip()
            else:
                s = '%'

    return s

In [9]:
lst = [clean(str(e)) for e in links] 

In [10]:
lista = [e for e in lst if '%' not in e]

In [11]:
lista

['http://wikipedia.org/wiki/Information_science',
 'http://wikipedia.org/wiki/Machine_learning',
 'http://wikipedia.org/wiki/Data_mining',
 'http://wikipedia.org/wiki/Statistical_classification',
 'http://wikipedia.org/wiki/Cluster_analysis',
 'http://wikipedia.org/wiki/Regression_analysis',
 'http://wikipedia.org/wiki/Anomaly_detection',
 'http://wikipedia.org/wiki/Automated_machine_learning',
 'http://wikipedia.org/wiki/Association_rule_learning',
 'http://wikipedia.org/wiki/Reinforcement_learning',
 'http://wikipedia.org/wiki/Structured_prediction',
 'http://wikipedia.org/wiki/Feature_engineering',
 'http://wikipedia.org/wiki/Feature_learning',
 'http://wikipedia.org/wiki/Online_machine_learning',
 'http://wikipedia.org/wiki/Semi-supervised_learning',
 'http://wikipedia.org/wiki/Unsupervised_learning',
 'http://wikipedia.org/wiki/Learning_to_rank',
 'http://wikipedia.org/wiki/Grammar_induction',
 'http://wikipedia.org/wiki/Supervised_learning',
 'http://wikipedia.org/wiki/Statistica

In [12]:
final = set(lista)
final = list(final)
final

['http://wikipedia.org/wiki/Anomaly_detection',
 'https://magazine.amstat.org/blog/2015/10/01/asa-statement-on-the-role-of-statistics-in-data-science/',
 'http://wikipedia.org/wiki/Factor_analysis',
 'http://wikipedia.org/wiki/Principal_component_analysis',
 'http://wikipedia.org/wiki/Automated_machine_learning',
 'http://wikipedia.org/wiki/Gated_recurrent_unit',
 'http://wikipedia.org/wiki/Statistical_classification',
 'http://wikipedia.org/wiki/U-Net',
 'http://wikipedia.org/wiki/List_of_datasets_for_machine-learning_research',
 'https://www.springer.com/book/9784431702085',
 'http://analytics-magazine.org/the-data-economy-why-do-so-many-analytics-projects-fail/',
 'http://ur.umich.edu/9899/Nov09_98/4.htm',
 'http://wikipedia.org/wiki/Jeffrey_T._Leek',
 'http://wikipedia.org/wiki/Random_forest',
 'http://wikipedia.org/wiki/K-nearest_neighbors_classification',
 'https://www.nytimes.com/2009/12/15/science/15books.html',
 'http://wikipedia.org/wiki/Linear_discriminant_analysis',
 'http:

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [13]:
import os

In [14]:
dirName = 'wikipedia'
 
try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")


Directory  wikipedia  Created 


In [15]:
os.chdir('wikipedia/')

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [16]:
from slugify import slugify

In [17]:
def index_page(s):
    try:
        html=str(requests.get(s).content)
        
        b = slugify(s) +'.html'
        file = open(b, "w")
        file.write(html)
        file.close()
        
              
        return 
    except:
        pass
    


## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [18]:
%%time 
for e in final:
    index_page(e)

Wall time: 3min 33s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [19]:
import multiprocessing

In [ ]:
pool = multiprocessing.Pool()
result = pool.map(index_page, final)
pool.terminate()
pool.join()
print(result)